In [ ]:
from sklearn.preprocessing import MinMaxScaler
minmax = MinMaxScaler()
true_stand_data=minmax.fit_transform(true_stand_data)
false_stand_data=minmax.fit_transform(false_stand_data)

In [ ]:
import keras
import pandas as pd
from keras.layers.core import Dense, Dropout
from keras.layers.recurrent import GRU
from keras.models import Sequential, load_model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import copy

from keras.layers.recurrent import GRU
from keras.models import Model
from keras.layers import Input, Convolution2D, Flatten, Dense, Concatenate,Dropout
import copy
import numpy as np
import tensorflow as tf

true_stand_data=np.load(r'.//datatarget//true_array_target.npy')
false_stand_data=np.load(r'.//datatarget//false_array_target.npy')

true_stand_data=true_stand_data[:,:,1:12]
false_stand_data=false_stand_data[:,:,1:12]

true_stand_data=true_stand_data.reshape((-1,11))
false_stand_data=false_stand_data.reshape((-1,11))
# true_stand_data= np.nan_to_num(true_stand_data)
# false_stand_data=np.nan_to_num(false_stand_data)

minmax = MinMaxScaler()
scaler=minmax.fit(true_stand_data)

true_stand_data=scaler.transform(true_stand_data)
false_stand_data=scaler.transform(false_stand_data)

true_stand_data=true_stand_data.reshape((-1,110,11))
false_stand_data=false_stand_data.reshape((-1,110,11))

sample_num={'true_num':true_stand_data.shape[0],'false_num':false_stand_data.shape[0]}
class_weight = {
                0: (1 / sample_num['false_num'] * (sample_num['true_num'] + sample_num['false_num'])) / 2,
                1: (1 / sample_num['true_num'] * (sample_num['true_num'] + sample_num['false_num'])) / 2
                }


true_sample_channel_1 = []
true_sample_channel_1_left = []
true_sample_channel_1_middle = []
true_sample_channel_1_right = []


false_sample_channel_1 = []
false_sample_channel_1_left = []
false_sample_channel_1_middle = []
false_sample_channel_1_right = []

for sample_number in range(true_stand_data.shape[0]):
    true_sample_channel_1.append(copy.deepcopy(true_stand_data[sample_number,:,:]))
    true_sample_channel_1_left.append(copy.deepcopy(true_stand_data[sample_number,-74:,:]))
    true_sample_channel_1_middle.append(copy.deepcopy(true_stand_data[sample_number,-38:,:]))
    true_sample_channel_1_right.append(copy.deepcopy(true_stand_data[sample_number,-20:,:]))

for sample_number in range(false_stand_data.shape[0]):
    false_sample_channel_1.append(copy.deepcopy(false_stand_data[sample_number,:,:]))
    false_sample_channel_1_left.append(copy.deepcopy(false_stand_data[sample_number,-74:,:]))
    false_sample_channel_1_middle.append(copy.deepcopy(false_stand_data[sample_number,-38:,:]))
    false_sample_channel_1_right.append(copy.deepcopy(false_stand_data[sample_number,-20:,:]))
    
combin_train_data1 = np.concatenate((true_sample_channel_1,false_sample_channel_1),axis=0)
combin_train_data1_left = np.concatenate((true_sample_channel_1_left,false_sample_channel_1_left),axis=0)
combin_train_data1_middle = np.concatenate((true_sample_channel_1_middle,false_sample_channel_1_middle),axis=0)
combin_train_data1_right = np.concatenate((true_sample_channel_1_right,false_sample_channel_1_right),axis=0)
    
combin_target_data= np.concatenate((np.ones((sample_num['true_num'],1)),np.zeros((sample_num['false_num'],1))),axis=0)
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder()
combin_target_data=ohe.fit_transform(combin_target_data).toarray()

# Model 1
in_1 = Input(shape=(110,11))
x_480_1 = GRU(units=480,return_sequences=True,input_shape=(110,11))(in_1)
x_480_1 =Dropout(0.1)(x_480_1)
x_48_1 = GRU(units=48)(x_480_1)

x_240_1 = GRU(units=240,return_sequences=True,input_shape=(110,11))(in_1)
x_240_1 =Dropout(0.1)(x_240_1)
x_24_1 = GRU(units=24)(x_240_1)

x_1_concat = Concatenate(axis=-1)([x_48_1,x_24_1])
x_1_concat_out = Dense(3, activation='tanh')(x_1_concat)

###########################
in_left = Input(shape=(74,11))
x_480_left = GRU(units=240,return_sequences=True,input_shape=(74,11))(in_left)
x_480_left =Dropout(0.1)(x_480_left)
x_48_left = GRU(units=24)(x_480_left)

x_240_left = GRU(units=120,return_sequences=True,input_shape=(74,11))(in_left)
x_240_left =Dropout(0.1)(x_240_left)
x_24_left = GRU(units=12)(x_240_left)

x_left_concat = Concatenate(axis=-1)([x_48_left,x_24_left])
x_left_concat_out = Dense(3, activation='tanh')(x_left_concat)

#############################
in_middle = Input(shape=(38,11))
x_480_middle = GRU(units=120,return_sequences=True,input_shape=(38,11))(in_middle)
x_480_middle =Dropout(0.1)(x_480_middle)
x_48_middle = GRU(units=12)(x_480_middle)

x_240_middle = GRU(units=60,return_sequences=True,input_shape=(38,11))(in_middle)
x_240_middle =Dropout(0.1)(x_240_middle)
x_24_middle = GRU(units=6)(x_240_middle)

x_middle_concat = Concatenate(axis=-1)([x_48_middle,x_24_middle])
x_middle_concat_out = Dense(3, activation='tanh')(x_middle_concat)


###########################
in_right = Input(shape=(20,11))
x_480_right = GRU(units=60,return_sequences=True,input_shape=(20,11))(in_right)
x_480_right =Dropout(0.1)(x_480_right)
x_48_right = GRU(units=6)(x_480_right)

x_240_right = GRU(units=30,return_sequences=True,input_shape=(20,11))(in_right)
x_240_right =Dropout(0.1)(x_240_right)
x_24_right = GRU(units=3)(x_240_right)

x_right_concat = Concatenate(axis=-1)([x_48_right,x_24_right])
x_right_concat_out = Dense(3, activation='tanh')(x_right_concat)

total_out_concat = Concatenate(axis=-1)([x_1_concat_out,x_left_concat_out,x_middle_concat_out,x_right_concat_out])
model_final_dense_out = Dense(2, activation='softmax')(total_out_concat)

model = Model(inputs=[in_1,in_left,in_middle,in_right], outputs=model_final_dense_out)
print(model.summary())

import os
if 'GRU_predict_v6.h5' in os.listdir('./'):
    model = load_model("./GRU_predict_v6.h5")

callbacks_list = [    
    keras.callbacks.EarlyStopping(
        monitor='accuracy', 
        patience=30
    ),
    # 保存模型
    keras.callbacks.ModelCheckpoint(
    filepath = 'GRU_predict_v6.h5', 
    monitor='loss', 
    save_best_only=True),
    
    keras.callbacks.ReduceLROnPlateau(
        monitor='loss', 
        factor=0.8,
        patience=5, 
        mode='auto',
        min_lr=0.00003)
]


model.compile(loss='categorical_crossentropy', #continu together
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              metrics=['accuracy']
             
             )

model.fit(x=[combin_train_data1,combin_train_data1_left,combin_train_data1_middle,combin_train_data1_right],
          y=combin_target_data,
          batch_size=360,
          epochs=100,
          shuffle=True,
          verbose=1,
          class_weight=class_weight,
          callbacks=callbacks_list      
         )

    